In [1]:
#works for Google Colab as at Feb 2026.

import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
%pip install selenium
!apt-get update
!apt install chromium-chromedriver
#!apt install chromium-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://r2u.stat

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
#%pip install webdriver_manager

In [4]:
# Update package list and install necessary dependencies
!apt-get update
!apt-get install -y wget unzip libvulkan1

# Download and install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

# Install xvfb for virtual framebuffer support
!apt-get install -y xvfb

# Install selenium and chromedriver-autoinstaller
!pip install selenium chromedriver-autoinstaller

# Automatically install the correct version of ChromeDriver
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
wget is already the newest version (1.2

'/usr/local/lib/python3.12/dist-packages/chromedriver_autoinstaller/144/chromedriver'

In [7]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [8]:
driver.get("https://summerofcode.withgoogle.com/archive/2016/projects/")

def extract_data(soup):

  all_a=soup.find_all('a')
  titles =soup.find_all(class_="title")
  descriptions=soup.find_all(class_ ="description")
  contibutors =soup.find_all(class_="contributor__content")
  mentors =soup.find_all(class_="mentor")
  organizations =soup.find_all(class_="organization")

  titles=[t.text if t is not None else "" for t in titles]
  descriptions=[d.text if d is not None else "" for d in descriptions]
  contibutors=[c.text if c is not None else "" for c in contibutors]
  mentors=[m.text if m is not None else "" for m in mentors]
  organizations=[o.text if o is not None else "" for o in organizations]


  archives=[]
  links=[]
  for a in all_a:
    arch=a.find(class_="mdc-button")
    p_a=a.get("class")
    if p_a is not None:
      if len(p_a)>1:
        if p_a[1]=="mdc-button--unelevated":
          archives.append(a.get('href'))
        elif p_a[1]=="mdc-button--outlined":
          links.append(a.get('href'))

  return titles,descriptions,contibutors,mentors,organizations,archives,links

#button =driver.find_element(By.CLASS_NAME,"mat-mdc-paginator-navigation-next").click()
time.sleep(10)
found = True
j=0
while found:
    output=pd.DataFrame(columns=["Id","Title","Description","Contributor","Mentor","Organization","Archive","Github"])
    try:
        # Attempt to get the page source
        #driver.get("https://summerofcode.withgoogle.com/archive/2024/projects/")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        titles,descriptions,contibutors,mentors,organizations,archives,links=extract_data(soup)
        j+=1
        #print(j)
        for i in range(len(titles)):
            result=[]
            result.append(i+1)
            lt=len(titles)
            ld=len(descriptions)
            lc=len(contibutors)
            lm=len(mentors)
            lo=len(organizations)
            la=len(archives)
            ll=len(links)

            if i<lt:
              result.append(titles[i])
            else:
              result.append("")
            if i<ld:
              result.append(descriptions[i])
            else:
              result.append("")
            if i<lc:
              result.append(contibutors[i])
            else:
              result.append("")
            if i<lm:
              result.append(mentors[i])
            else:
              result.append("")
            if i<lo:
              result.append(organizations[i])
            else:
              result.append("")
            if i<la:
              result.append(archives[i])
            else:
              result.append("")
            if i<ll:
              result.append(links[i])
            else:
              result.append("")
            #output.loc[i]=[i+1,titles[i],descriptions[i],contibutors[i],mentors[i],organizations[i],archives[i],links[i]]
            #print(i)
            output.loc[i]=result
        print(j)
        #if j>1:
        #   found=False

        time.sleep(10)
        name="summer_code_2016_"+str(j)+".csv"
        output.to_csv(name,index=False)
        #print(j)

        button =driver.find_element(By.CLASS_NAME,"mat-mdc-paginator-navigation-next")
        #tag_name= button.get_attribute("tagName").lower()
        #print(tag_name)
        #print("initial",button)
        #time.sleep(5)
        #found=False
        #buttons =driver.find_elements(By.CLASS_NAME,"mat-mdc-paginator-navigation-next")
        #for b in buttons:
        #  print("original", b)
        #  if b.is_displayed() and b.is_enabled():
         #   print("clickable ",b)
            #b.click()
        #    found=True
        #    print("clicked")

          #print("buttons")
          #print(b)

        #print("button: ",button)
        driver.execute_script("arguments[0].click();", button)
        #button.click()
        print("test 2")


        if button.get_attribute("disabled"):
            found = False
        time.sleep(10)
    except Exception as e:
        print (e)
        print(type(e))
        print("error occurred")
        print("error",j)

        # Check if the error is a connection issue with the WebDriver
        from urllib3.exceptions import MaxRetryError
        from selenium.common.exceptions import WebDriverException # Generic Selenium exception

        if isinstance(e, MaxRetryError) or (isinstance(e, ConnectionRefusedError)) or (isinstance(e, WebDriverException) and "connection refused" in str(e).lower()):
            print("WebDriver connection lost. Attempting to restart WebDriver...")
            try:
                driver.quit() # Try to clean up the old driver instance
            except:
                pass # Ignore if quit() fails
            # Re-initialize the driver (assuming chrome_options is still available from cell KTrpR97aBPrF)
            driver = webdriver.Chrome(options=chrome_options)
            # Set found to True to retry the current page after restarting the driver
            found = True
            # It might be good to add a small sleep here to allow the driver to stabilize
            time.sleep(5)
            continue # Continue to the next iteration of the loop to retry
        else:
            name="summer_code_2016_"+str(j)+".csv"
            output.to_csv(name,index=False)
            found = False # For other types of errors, stop the loop

soup = BeautifulSoup(driver.page_source, 'html.parser')
titles,descriptions,contibutors,mentors,organizations,archives,links=extract_data(soup)
output=pd.DataFrame(columns=["Id","Title","Description","Contributor","Mentor","Organization","Archive","Github"])
for i in range(len(titles)):
            result=[]
            result.append(str(j))
            if titles[i] is not None:
              result.append(titles[i])
            else:
              result.append("")
            if descriptions[i] is not None:
              result.append(descriptions[i])
            else:
              result.append("")
            if contibutors[i] is not None:
              result.append(contibutors[i])
            else:
              result.append("")
            if mentors[i] is not None:
              result.append(mentors[i])
            else:
              result.append("")
            if organizations[i] is not None:
              result.append(organizations[i])
            else:
              result.append("")
            if archives[i] is not None:
              result.append(archives[i])
            else:
              result.append("")
            if links[i] is not None:
              result.append(links[i])
            else:
              result.append("")
            #output.loc[i]=[i+1,titles[i],descriptions[i],contibutors[i],mentors[i],organizations[i],archives[i],links[i]]
            output.loc[i]=result
#output.loc[i]=[i+1,titles[i],descriptions[i],contibutors[i],mentors[i],organizations[i],archives[i],links[i]]
name="summer_code_2016_last.csv"
output.to_csv(name,index=False)

driver.quit()


1
test 2
2
test 2
3
test 2
4
test 2
5
test 2
6
test 2
7
test 2
8
test 2
9
test 2
10
test 2
11
test 2
12
test 2
13
test 2
14
test 2
15
test 2
16
test 2
17
test 2
18
test 2
19
test 2
20
test 2


In [14]:
driver.quit()